<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1b
# *Data Cleaning with Pandas*

In this lab we work with a file that has messy column names and a combination of missing and invalid data to be cleaned.

To start with load `rock.csv` into a dataframe.

In [494]:
import pandas as pd
df = pd.read_csv(r'/Users/annaxu/Downloads/DATA/rock.csv')
df.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


### Check Column Names

Check column names and clean by converting all names to lowercase, replacing spaces with underscores (`_`) and removing question marks or asterisk (`*`) characters.

In [497]:
df.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

In [499]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_') 
df.columns = df.columns.str.replace(r'[?,*]', '', regex=True)
print(df.columns)

Index(['song_clean', 'artist_clean', 'release_year', 'combined', 'first',
       'year', 'playcount', 'fg'],
      dtype='object')


### Replace Null Values With 0

Check 'release_year' column whether this column has any null values or not. Replace null values with 0.

In [502]:
df['release_year'].isnull().value_counts()

release_year
False    1653
True      577
Name: count, dtype: int64

In [504]:
df['release_year'] = df['release_year'].fillna(0)
df['release_year'].isnull().value_counts()

release_year
False    2230
Name: count, dtype: int64

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column.

In [507]:
df.dtypes

song_clean      object
artist_clean    object
release_year    object
combined        object
first            int64
year             int64
playcount        int64
fg               int64
dtype: object

In [509]:
df['release_year'].astype('int64')

ValueError: invalid literal for int() with base 10: 'SONGFACTS.COM'

In [511]:
print(df[df['release_year'] == 'SONGFACTS.COM'])

          song_clean    artist_clean   release_year  \
1504  Bullfrog Blues  Rory Gallagher  SONGFACTS.COM   

                              combined  first  year  playcount  fg  
1504  Bullfrog Blues by Rory Gallagher      1     1          1   1  


In [513]:
df.loc[df['release_year'] == 'SONGFACTS.COM', 'release_year'] = 0

In [515]:
df['release_year'] = df['release_year'].astype('int64')

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [518]:
df.describe().loc[['min', 'max']]

,release_year,first,year,playcount,fg
min,0.0,1.0,0.0,0.0,0.0
max,2014.0,1.0,1.0,142.0,142.0


In [520]:
df['release_year'].value_counts().sort_index().head(14)

release_year
0       578
1071      1
1955      1
1958      1
1961      1
1962      3
1963      9
1964     14
1965     28
1966     30
1967     61
1968     46
1969     72
1970     81
Name: count, dtype: int64

In [522]:
#Drop rows where release_year = 0 because it is a significant portion of the dataset so it would heavily skew the data if we were to replace with the median or mean.
#Drop release_year = 1071 as it's an outlier.
df = df.drop(df[(df['release_year'] == 0) | (df['release_year'] == 1071)].index)
df.shape

(1651, 8)

## Write Some Functions

### Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [526]:
def check_release_year(df):
    df = df.copy()
    df["release_before_1970"] = df["release_year"].apply(lambda x: "Yes" if x < 1970 else "No")
    return df[["song_clean", "artist_clean", "release_before_1970"]]
    
new_df = check_release_year(df)
print(new_df)

                                       song_clean  artist_clean  \
0                                Caught Up in You   .38 Special   
2                                 Hold On Loosely   .38 Special   
3                          Rockin' Into the Night   .38 Special   
4                               Art For Arts Sake          10cc   
5                                      Kryptonite  3 Doors Down   
...                                           ...           ...   
2224                            Sharp Dressed Man        ZZ Top   
2226                            Tube Snake Boogie        ZZ Top   
2227                                         Tush        ZZ Top   
2228                                   TV Dinners        ZZ Top   
2229  WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO        ZZ Top   

     release_before_1970  
0                     No  
2                     No  
3                     No  
4                     No  
5                     No  
...                  ...  
2224  

In [528]:
(new_df['release_before_1970'] == 'Yes').sum()

266

### Write a function that converts a column in a DataFrame to a numeric type and otherwise replaces entries with np.nan

In [486]:
def convert_to_numeric(df, column_name):
    df = df.copy()
    df[column_name] = pd.to_numeric(df[column_name], errors="coerce")
    return df

### Apply this last function to your dataset

In [568]:
#Create a random column with mixed data
import random
possible_values = ["123", "45.67", "NaN", "abc", "2024", "$56", "7.89", "?", "1000", "None"]
df["test_column"] = [random.choice(possible_values) for _ in range(len(df))]
print(df['test_column'].head())
print(df.dtypes)

0      $56
2    45.67
3     None
4        ?
5      NaN
Name: test_column, dtype: object
song_clean      object
artist_clean    object
release_year     int64
combined        object
first            int64
year             int64
playcount        int64
fg               int64
test_column     object
dtype: object


In [572]:
df = convert_to_numeric(df, 'test_column')
print(df['test_column'].head())
print(df.dtypes)

0      NaN
2    45.67
3      NaN
4      NaN
5      NaN
Name: test_column, dtype: float64
song_clean       object
artist_clean     object
release_year      int64
combined         object
first             int64
year              int64
playcount         int64
fg                int64
test_column     float64
dtype: object


### 'Describe' the new DataFrame

In [574]:
df.describe()

,release_year,first,year,playcount,fg,test_column
count,1651.000000,1651.0,1651.0,1651.000000,1651.000000,818.000000
mean,1978.569352,1.0,1.0,20.320412,20.320412,623.306002
std,9.309780,0.0,0.0,27.509222,27.509222,780.579435
min,1955.000000,1.0,1.0,0.000000,0.000000,7.890000
25%,1971.000000,1.0,1.0,2.000000,2.000000,45.670000
50%,1977.000000,1.0,1.0,7.000000,7.000000,123.000000
75%,1984.000000,1.0,1.0,28.000000,28.000000,1000.000000
max,2014.000000,1.0,1.0,142.000000,142.000000,2024.000000


>




---



---



> > > > > > > > > © 2025 Institute of Data


---



---



